## **Fase 2 - Limpieza de Datos**
*En esta fase crucial del proceso ETL, la limpieza de datos se convierte en la base para un análisis confiable y preciso. La limpieza implica la eliminación de valores nulos, la corrección de incoherencias y el tratamiento de datos que podrían distorsionar los resultados. Este proceso garantiza que el conjunto de datos esté libre de ruido y preparado para extraer información valiosa. Al limpiar los datos, aseguramos que las conclusiones posteriores reflejen fielmente la realidad representada en los datos, estableciendo una base sólida sobre la cual se construirá nuestro análisis*.

In [3]:
# General
import numpy as np
import pandas as pd
from datetime import datetime
import re

In [4]:
df = pd.read_csv('data/df_extract.csv')

### **Análisis inicial del dataset para entender la estructura, los tipos de datos, los valores ausentes y las estadísticas generales**

In [6]:
df.head()

,urls,timestamp,record_id,Titles,Host_name,Property_types,Prices_per_night,Check_ins,Check_outs,Location,Ratings,Num_reviews,Cleaning_fee,Com_fee,Viajeros,Dormitorios,Camas,Baños,Maximum_guests,Otros
0,https://www.airbnb.es/rooms/126311759129279497...,2024-11-07 23:54:16,1263117591292794971,"La ubicación muy buena, cerca de todo si estás...",Anfitrión: Carmen,"Alojamiento entero: apartamento en Barcelona, ...",115 €,Llegada a partir de las 15:00,Salida antes de las 12:00,"Barcelona, Catalunya, España",Nuevo,NaN,0,0,3 viajeros,1 dormitorio,1 cama,1 baño,NaN,NaN
1,https://www.airbnb.es/rooms/127904085557632410...,2024-11-07 23:54:16,1279040855576324107,Este anfitrión tiene 979 evaluaciones de otros...,Anfitrión: David,"Habitación en Barcelona, España",46 €,Horario de llegada: de 17:00 a 2:00,Salida antes de las 11:00,"Barcelona, Catalunya, España",Nuevo,NaN,15 €,42 €,NaN,NaN,1 cama,Baño compartido,1.0,NaN
2,https://www.airbnb.es/rooms/913187918206344111...,2024-11-07 23:54:16,913187918206344111,Esta acogedora habitación cuenta con una cómo...,Quédate con Manuel,"Habitación en Barcelona, España",47 €,Llegada a partir de las 15:00,Salida antes de las 12:00,NaN,"4,66",65.0,0,40 €,NaN,NaN,1 cama,Baño compartido,1.0,NaN
3,https://www.airbnb.es/rooms/126566083301808951...,2024-11-07 23:54:16,1265660833018089515,Instalaciones y servicios nuevos a estrenar.Es...,Anfitrión: BLAU Student Housing,"Alojamiento entero: apartamento en Barcelona, ...",100 €,Llegada a partir de las 16:00,Salida antes de las 12:00,"Barcelona, Catalunya, España","4,89",NaN,35 €,0,1 viajero,1 dormitorio,1 cama,1 baño,1.0,NaN
4,https://www.airbnb.es/rooms/31977850?adults=1&...,2024-11-07 23:54:16,31977850,¡Hay una razón por la que hemos sido votados c...,Anfitrión: St Christophers Barcelona,"Habitación en Barcelona, España",33 €,Llegada a partir de las 15:00,Salida antes de las 11:00,"Barcelona, España","4,4",990.0,0,0,NaN,NaN,1 cama individual,Baño compartido,1.0,NaN


In [7]:
df.shape

(2700, 20)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700 entries, 0 to 2699
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   urls              2700 non-null   object 
 1   timestamp         2700 non-null   object 
 2   record_id         2700 non-null   int64  
 3   Titles            2694 non-null   object 
 4   Host_name         2694 non-null   object 
 5   Property_types    2694 non-null   object 
 6   Prices_per_night  2526 non-null   object 
 7   Check_ins         2694 non-null   object 
 8   Check_outs        2694 non-null   object 
 9   Location          1379 non-null   object 
 10  Ratings           2676 non-null   object 
 11  Num_reviews       1838 non-null   float64
 12  Cleaning_fee      2700 non-null   object 
 13  Com_fee           2700 non-null   object 
 14  Viajeros          1458 non-null   object 
 15  Dormitorios       1470 non-null   object 
 16  Camas             2585 non-null   object 


In [9]:
df.describe()

,record_id,Num_reviews,Maximum_guests
count,2.700000e+03,1838.000000,2693.000000
mean,6.268210e+17,183.539173,2.823245
std,5.432747e+17,315.762477,1.643559
min,7.243600e+04,2.000000,1.000000
25%,3.197785e+07,24.000000,2.000000
50%,8.361664e+17,79.000000,2.000000
75%,1.159331e+18,214.000000,4.000000
max,1.280281e+18,2782.000000,11.000000


In [10]:
df.isna().sum()

urls                   0
timestamp              0
record_id              0
Titles                 6
Host_name              6
Property_types         6
Prices_per_night     174
Check_ins              6
Check_outs             6
Location            1321
Ratings               24
Num_reviews          862
Cleaning_fee           0
Com_fee                0
Viajeros            1242
Dormitorios         1230
Camas                115
Baños                 73
Maximum_guests         7
Otros               2535
dtype: int64

### **Ponemos todas las columnas en minúsculas**

In [12]:
df.columns = df.columns.str.lower()
df.head()

,urls,timestamp,record_id,titles,host_name,property_types,prices_per_night,check_ins,check_outs,location,ratings,num_reviews,cleaning_fee,com_fee,viajeros,dormitorios,camas,baños,maximum_guests,otros
0,https://www.airbnb.es/rooms/126311759129279497...,2024-11-07 23:54:16,1263117591292794971,"La ubicación muy buena, cerca de todo si estás...",Anfitrión: Carmen,"Alojamiento entero: apartamento en Barcelona, ...",115 €,Llegada a partir de las 15:00,Salida antes de las 12:00,"Barcelona, Catalunya, España",Nuevo,NaN,0,0,3 viajeros,1 dormitorio,1 cama,1 baño,NaN,NaN
1,https://www.airbnb.es/rooms/127904085557632410...,2024-11-07 23:54:16,1279040855576324107,Este anfitrión tiene 979 evaluaciones de otros...,Anfitrión: David,"Habitación en Barcelona, España",46 €,Horario de llegada: de 17:00 a 2:00,Salida antes de las 11:00,"Barcelona, Catalunya, España",Nuevo,NaN,15 €,42 €,NaN,NaN,1 cama,Baño compartido,1.0,NaN
2,https://www.airbnb.es/rooms/913187918206344111...,2024-11-07 23:54:16,913187918206344111,Esta acogedora habitación cuenta con una cómo...,Quédate con Manuel,"Habitación en Barcelona, España",47 €,Llegada a partir de las 15:00,Salida antes de las 12:00,NaN,"4,66",65.0,0,40 €,NaN,NaN,1 cama,Baño compartido,1.0,NaN
3,https://www.airbnb.es/rooms/126566083301808951...,2024-11-07 23:54:16,1265660833018089515,Instalaciones y servicios nuevos a estrenar.Es...,Anfitrión: BLAU Student Housing,"Alojamiento entero: apartamento en Barcelona, ...",100 €,Llegada a partir de las 16:00,Salida antes de las 12:00,"Barcelona, Catalunya, España","4,89",NaN,35 €,0,1 viajero,1 dormitorio,1 cama,1 baño,1.0,NaN
4,https://www.airbnb.es/rooms/31977850?adults=1&...,2024-11-07 23:54:16,31977850,¡Hay una razón por la que hemos sido votados c...,Anfitrión: St Christophers Barcelona,"Habitación en Barcelona, España",33 €,Llegada a partir de las 15:00,Salida antes de las 11:00,"Barcelona, España","4,4",990.0,0,0,NaN,NaN,1 cama individual,Baño compartido,1.0,NaN


#### **Eliminación de columnas redundantes o duplicadas que no aportan valor al proceso de análisis**

In [14]:
df = df.drop(['viajeros', 'otros', 'com_fee'], axis=1)

#### **Limpieza y Formateo de Columnas del DataFrame**

#### Titles

In [17]:
df['titles'] = df['titles'].str.capitalize()
# 2. Eliminar puntuación innecesaria
df['titles'] = df['titles'].str.replace('¡', '', regex=False)
# 3. Corregir espacios y saltos de línea
df['titles'] = df['titles'].str.replace(r'\s+', ' ', regex=True)
df['titles'] = df['titles'].str.strip()


#### Host

In [19]:
# Extraer solo el nombre del anfitrión, removendo "Anfitrión: " y cualquier outro prefixo
df['host_name'] = df['host_name'].str.replace('Anfitrión: ', '', regex=False).str.replace('Quédate con ', '', regex=False)

#### Property types

In [21]:
# Clasificamos los tipos de propiedad en tres categorías para extraer solo la información que queremos con una función
def category_types(value):
    if isinstance(value, float) and pd.isna(value):
        return "otro"
    elif "Alojamiento entero" in value:
        return "Alojamiento entero"
    elif "Habitación" in value:
        return "Habitación"
    else:
        return "otro"
df["property_types"] = df["property_types"].apply(category_types)    

#### Prices

In [23]:
# Remover el símbolo € y convertir a valores numéricos
df['prices_per_night'] = pd.to_numeric(df['prices_per_night'].str.replace('€', '').str.strip(), errors='coerce')
df = df.dropna(subset=['prices_per_night']) # Mantener solo registros válidos para análisis

In [24]:
# Verificación después del dropna
df.shape

(2526, 17)

#### Check in/Check out

In [26]:
def process_check_times(df):

    def extract_check_in_hour(check_in):

        if isinstance(check_in, str):
            check_in = check_in.strip().lower()

            if 'llegada' in check_in:
                # Elimina la parte "llegada" y obtiene la primera hora válida
                check_in = check_in.replace('llegada:', '')
                check_in = check_in.replace('de', '')
                parts = check_in.split()
                for part in parts:
                    if ":" in part:  # Verifica si el elemento tiene formato de hora "HH:MM"
                        return part.strip()

        return np.nan

    def extract_check_out_hour(check_out):

        if isinstance(check_out, str) and "antes de las" in check_out:
            try:  # Extrae solo la hora
                hour_str = check_out.split("antes de las")[1].split(":")[0].strip() + ":00"  # Devuelve la hora en formato "HH:00"
                return hour_str
            except:
                return np.nan
        return np.nan

    # Función para calcular las horas restantes hasta el check-in
    def calculate_hours_to_check_in(check_in_hour):
        try:
            # Extraímos la hora directamente de la cadena
            hour = int(check_in_hour.split(":")[0])  # Obtiene la hora (antes de los minutos)

            # Verifica si la hora es válida
            if 0 <= hour < 24:
                return 24 - hour  # Calcula las horas restantes hasta el check-in
        except Exception:
            return np.nan

        return np.nan

    # Aplica las funciones a las columnas correspondientes
    df['check_in_hour'] = df['check_ins'].apply(extract_check_in_hour)
    df['check_out_hour'] = df['check_outs'].apply(extract_check_out_hour)
    df['total_hours_checkin'] = df['check_in_hour'].apply(calculate_hours_to_check_in)

    df = df.drop(columns=['check_ins', 'check_outs'])

    return df


In [27]:
df = process_check_times(df)

#### Cleaning Fees

In [29]:
df['cleaning_fee'] = pd.to_numeric(df['cleaning_fee'].str.replace('€', '').str.strip(), errors='coerce')

#### Ratings

In [31]:
# Sustituir valores "Nuevo" y "Sin evaluaciones" por 0
df['ratings'] = df['ratings'].replace({'Nuevo': '0', 'Sin evaluaciones': '0'})

# Eliminar comas y convertir a numérico
df['ratings'] = pd.to_numeric(df['ratings'].str.replace(',', '.'), errors='coerce').fillna(0)

#### Number of reviews

In [33]:
# Convertir a numérico y reemplazar valores nulos con 0
df['num_reviews'] = pd.to_numeric(df['num_reviews'], errors='coerce').fillna(0)

#### Dormitorios

In [35]:
# Extraer solo el número de dormitorios eliminando letras
df['dormitorios'] = df['dormitorios'].str.extract('(\d+)')[0]

df['dormitorios'] = pd.to_numeric(df['dormitorios'], errors='coerce')

#### Camas

In [37]:
# Extraer solo el número de camas eliminando letras
df['camas'] = df['camas'].str.extract('(\d+)')[0]

df['camas'] = pd.to_numeric(df['camas'], errors='coerce')

#### Baños

In [39]:
# Función adaptada para la columna 'baños'
def extract_bathrooms(value):
    # Primero, intentamos encontrar un número en el valor
    match = re.search(r'\d+(\.\d+)?', str(value))  # Busca números enteros o decimales
    if match:
        # Si se encuentra un número, devolverlo como float
        return float(match.group())
    else:
        # Si no se encuentra un número, asignamos valores predeterminados basados en palabras clave
        value = str(value).lower()  # Convertir a minúsculas para mejor coincidencia
        if 'compartido' in value:
            return 0.5
        elif 'privado' in value:
            return 1.0
        elif 'sin baño' in value or '0 baños' in value:
            return 0.0
        else:
            return None

In [40]:
df['baños'] = df['baños'].apply(extract_bathrooms)

In [41]:
df.to_csv('data/df_final_cleaned.csv', index=False)

# Limpieza de servicios

### Vision general

In [44]:
df_servicios = pd.read_csv('data/df_servicios_extract.csv')

In [45]:
df_servicios.head()

,Category,Service,url
0,Baño,Secador de pelo,https://www.airbnb.es/rooms/126311759129279497...
1,Baño,Productos de limpieza,https://www.airbnb.es/rooms/126311759129279497...
2,Baño,Champú,https://www.airbnb.es/rooms/126311759129279497...
3,Baño,Gel de ducha,https://www.airbnb.es/rooms/126311759129279497...
4,Baño,Agua caliente,https://www.airbnb.es/rooms/126311759129279497...


In [46]:
df_servicios.shape

(90749, 3)

In [47]:
df_servicios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90749 entries, 0 to 90748
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  90749 non-null  object
 1   Service   90749 non-null  object
 2   url       90749 non-null  object
dtypes: object(3)
memory usage: 2.1+ MB


In [48]:
df_servicios.describe()

,Category,Service,url
count,90749,90749,90749
unique,15,1279,2669
top,Cocina y comedor,Wifi,https://www.airbnb.es/rooms/829229988526268828...
freq,21221,2416,80


In [49]:
df_servicios.columns

Index(['Category', 'Service', 'url'], dtype='object')

In [50]:
df_servicios.isna().sum()

Category    0
Service     0
url         0
dtype: int64

### Cambiamos el nombre de la columna url a urls

In [52]:
df_servicios= df_servicios.rename(columns = {'url' : 'urls'})

### Cambiamos todas las columnas a minuscula ###

In [54]:
df_servicios.columns = df_servicios.columns.str.lower()
df_servicios

,category,service,urls
0,Baño,Secador de pelo,https://www.airbnb.es/rooms/126311759129279497...
1,Baño,Productos de limpieza,https://www.airbnb.es/rooms/126311759129279497...
2,Baño,Champú,https://www.airbnb.es/rooms/126311759129279497...
3,Baño,Gel de ducha,https://www.airbnb.es/rooms/126311759129279497...
4,Baño,Agua caliente,https://www.airbnb.es/rooms/126311759129279497...
...,...,...,...
90744,Servicios,Llegada autónoma,https://www.airbnb.es/rooms/14352069?adults=1&...
90745,Servicios,Caja de seguridad para llaves,https://www.airbnb.es/rooms/14352069?adults=1&...
90746,No incluidos,No disponible: Detector de monóxido de carbono...,https://www.airbnb.es/rooms/14352069?adults=1&...
90747,No incluidos,No disponible: ChampúChampú,https://www.airbnb.es/rooms/14352069?adults=1&...


### Eliminar posibles filas duplicadas ###

In [56]:
# Identificar filas duplicadas
duplicated_rows = df_servicios[df_servicios.duplicated(keep=False)]
duplicated_rows

,category,service,urls
3,Baño,Gel de ducha,https://www.airbnb.es/rooms/126311759129279497...
5,Baño,Gel de ducha,https://www.airbnb.es/rooms/126311759129279497...
43,Baño,Gel de ducha,https://www.airbnb.es/rooms/127904085557632410...
45,Baño,Gel de ducha,https://www.airbnb.es/rooms/127904085557632410...
77,Baño,Gel de ducha,https://www.airbnb.es/rooms/913187918206344111...
...,...,...,...
90334,Baño,Gel de ducha,https://www.airbnb.es/rooms/1790446?adults=1&c...
90378,Baño,Gel de ducha,https://www.airbnb.es/rooms/923118867334043435...
90381,Baño,Gel de ducha,https://www.airbnb.es/rooms/923118867334043435...
90655,Baño,Gel de ducha,https://www.airbnb.es/rooms/52528580?adults=1&...


In [57]:
# Test para ver los datos duplicados de algunas filas
df_servicios['urls'][15934]

'https://www.airbnb.es/rooms/1219487292461143358?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1990210809&search_mode=regular_search&check_in=2024-11-17&check_out=2024-11-22&source_impression_id=p3_1731019452_P3UbTZj75L6PH9FG&previous_page_section_name=1000&federated_search_id=36eb50cd-e222-4e00-b93f-e4e4f82fb568'

In [58]:
# Limpiar duplicados (creamos una variable para no perder el original)
df_servicios_cleaned = df_servicios.drop_duplicates()
df_servicios_cleaned

,category,service,urls
0,Baño,Secador de pelo,https://www.airbnb.es/rooms/126311759129279497...
1,Baño,Productos de limpieza,https://www.airbnb.es/rooms/126311759129279497...
2,Baño,Champú,https://www.airbnb.es/rooms/126311759129279497...
3,Baño,Gel de ducha,https://www.airbnb.es/rooms/126311759129279497...
4,Baño,Agua caliente,https://www.airbnb.es/rooms/126311759129279497...
...,...,...,...
90744,Servicios,Llegada autónoma,https://www.airbnb.es/rooms/14352069?adults=1&...
90745,Servicios,Caja de seguridad para llaves,https://www.airbnb.es/rooms/14352069?adults=1&...
90746,No incluidos,No disponible: Detector de monóxido de carbono...,https://www.airbnb.es/rooms/14352069?adults=1&...
90747,No incluidos,No disponible: ChampúChampú,https://www.airbnb.es/rooms/14352069?adults=1&...


In [59]:
df_servicios_cleaned.shape

(90041, 3)

### Limpiar formato de texto ###

In [61]:
# Ponemos los datos de forma que no haya espacios y en minuscula para facilitar manipulacion
# Sale un warning de copia, para evitarlo se usa .loc --> df_servicios_cleaned.loc[:,'Category'] = df_servicios_cleaned['Category'].str.strip().str.lower()

df_servicios_cleaned['category'] = df_servicios_cleaned['category'].str.strip().str.lower()
df_servicios_cleaned['service'] = df_servicios_cleaned['service'].str.strip().str.lower()

C:\Users\jesus\AppData\Local\Temp\ipykernel_20116\2199915870.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_servicios_cleaned['category'] = df_servicios_cleaned['category'].str.strip().str.lower()
C:\Users\jesus\AppData\Local\Temp\ipykernel_20116\2199915870.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_servicios_cleaned['service'] = df_servicios_cleaned['service'].str.strip().str.lower()


In [62]:
df_servicios_cleaned.head()

,category,service,urls
0,baño,secador de pelo,https://www.airbnb.es/rooms/126311759129279497...
1,baño,productos de limpieza,https://www.airbnb.es/rooms/126311759129279497...
2,baño,champú,https://www.airbnb.es/rooms/126311759129279497...
3,baño,gel de ducha,https://www.airbnb.es/rooms/126311759129279497...
4,baño,agua caliente,https://www.airbnb.es/rooms/126311759129279497...


### Ver distribucion de servicios y mayor depuracion

In [64]:
df_servicios_cleaned['category'].value_counts()

category
cocina y comedor                   21221
dormitorio y lavandería            14916
no incluidos                       10765
baño                                9436
servicios                           5104
calefacción y refrigeración         4526
privacidad y seguridad              4333
internet y oficina                  4084
aparcamiento e instalaciones        3309
entretenimiento                     2808
seguridad en el hogar               2689
para familias                       2533
exterior                            2362
características de la ubicación     1360
vistas panorámicas                   595
Name: count, dtype: int64

In [65]:
df_servicios_cleaned['category'].nunique()

15

In [66]:
df_servicios_cleaned['service'].value_counts()

service
wifi                                                                                                                                                                   2416
agua caliente                                                                                                                                                          2238
cocinacocina disponible para el uso de los huéspedes                                                                                                                   2214
secador de pelo                                                                                                                                                        2058
perchas                                                                                                                                                                1923
                                                                                                                                    

In [67]:
df_servicios_cleaned['service'].nunique()

1256

In [68]:
# Crear un diccionario para almacenar los resultados de value_counts para cada categoría
category_service_counts = {}

# Iterar sobre cada categoría única
for category in df_servicios_cleaned['category'].unique():
    
    # Filtrar el DataFrame para la categoría actual y contar los servicios
    service_counts = df_servicios_cleaned[df_servicios_cleaned['category'] == category]['service'].value_counts()
    
    # Almacenar el resultado en el diccionario
    category_service_counts[category] = service_counts

# Ver resultados para una categoría, por ejemplo, "baño"
print(category_service_counts)


{'baño': service
agua caliente                         2238
secador de pelo                       2058
gel de ducha                          1437
champú                                1290
productos de limpieza                 1123
                                      ... 
gel de ducha de común                    1
champú de común                          1
champú de blanc                          1
champú de blanca                         1
gel de ducha de gel auchan generic       1
Name: count, Length: 131, dtype: int64, 'dormitorio y lavandería': service
perchas                                                                           1923
plancha                                                                           1812
servicios básicostoallas, sábanas, jabón y papel higiénico                        1812
ropa de cama                                                                      1372
persianas o cortinas opacas                                                       1227


In [69]:
cleaning_rules = {
    "baño": {
        "gel de ducha": "gel de ducha",
        "acondicionador": "acondicionador",
        "champú": "champú"
    },
    
    "dormitorio y lavandería": {
        "servicios básicostoallas" : "servicios básicos" ,
        "espacio para guardar la ropa" : "espacio para guardar la ropa",
        "ropa de cama" : "ropa de cama",
        "lavadora" : "lavadora",
        "secadora" : "secadora"
        
    },
    
    "entretenimiento" : {
        "televisión" : "tv",
        "equipo para hacer ejercicio" : "equipo para hacer ejercicio",
        "sistema de sonido" : "sistema de sonido",
        "habitación temática" : "habitación temática",
        "videoconsola" : "videoconsola"
    },

    "calefacción y refrigeración" : {
        "aire acondicionado" : "aire acondicionado",
        "calefacción" : "calefacción",
        "chimenea" : "chimenea",
        "ventilador" : "ventilador"
    },

    "privacidad y seguridad" : {
        #"no disponible" : "dropear" ****,
        "cámaras de seguridad" : "cámaras de seguridad",
        
    },

    "internet y oficina" : {
        "wifi" : "wifi",
        "zona para trabajar" : "zona para trabajar"
    },

    "cocina y comedor" : {
        "frigorífico" : "frigorífico",
        "platos y cubiertos" : "platos y cubiertos",
        "utensilios básicos" : "utensilios básicos",
        "cocina" : "cocina",
        "horno" : "horno",
        "cafetera" : "cafetera",
        "fogón" : "cocina",
        "utensilios de barbacoa" : "utensilios de barbacoa",
        "kitchenetteun" : "microondas"
        
    },

    "características de la ubicación" : {
        #"acceso a la playa" : "acceso a la playa" ****,
        "acceso compartido a la playa" : "acceso a la playa",
        "acceso al lago" : "no disponible",
        "pista de esquí" : "no disponible",
        "entrada independiente" : "entrada independiente",
        "acceso a la playa" : "acceso a la playa",
        "costajusto" : "al lado de una massa de agua",
        "complejo" : "no disponible",
        "resort" : "no disponible"
        
    },

    "aparcamiento e instalaciones" : {
        "ascensor" : "ascensor",
        "aparcamiento" : "aparcamiento",
        "gimnasio" : "gimnasio",
        "piscina" : "piscina",
        "jacuzzi" : "jacuzzi",
        "cargador para coches eléctricos" : "cargador para vehículos eléctricos",
        "cargador para vehículos eléctricos" : "cargador para vehículos eléctricos",
        "alojamiento de una alturaalojamiento sin escaleras" : "no disponible",
        "salón privado" : "no disponible"
    },

    "servicios" : {
        "servicio de limpieza" : "servicio de limpieza",
        "se permite dejar el equipaje" : "se permite dejar el equipaje",
        "desayuno" : "desayuno incluido",
        "personal en el edificio" : "personal en el edificio",
        "limpieza disponible durante la estancia" : "servicio de limpieza",
        "admite mascotas" : "admite mascotas",
        "cerradura con tecladoaccede" : "no disponible",
        "cerradura inteligente" : "no disponible"
    },

    # No incluidos --> hacer drop


    "para familias" : {
        "cuna" : "cuna",
        "trona" : "trona",
        "parque/cunas de viaje" : "parque/cunas de viaje",
        "bañera para bebés" : "bañera para bebés",
        "libros y juguetes" : "libros y juguetes", 
        "mesa para cambiar pañales" : "mesa para cambiar pañales",
        "sala de juegos infantil" : "libros y juguetes",
        "recomendaciones de canguros" : "no disponible",
        "parque infantil al aire libre" : "no disponible",
        "bicicletas para niños" : "no disponible",
        "monitor de bebés" : "no disponible"
    },

    "seguridad en el hogar" : {
        "humo" : "detector de humo",
        "cámaras de seguridad" : "cámaras de seguridad",
        "monitores de decibelios" : "monitores de decibelios",
        "detector de monóxido" : "detector de humo"
            
    },

    "exterior" : {
        "patio o balcón" : "patio o balcón",
        "patio trasero" : "patio",
        "barbacoa" : "barbacoa",
        "patio privado o balcón" : "patio o balcón",
        "Cocina exterior privada" : "no disponible",
        "kayak" : "no disponible",
        "cocina exterior" : "no disponible",
        "brasero exterior" : "no disponible",
        "atracadero" : "no disponible",
        "bicicletas" : "no disponible",
        "hamaca" : "no disponible",
        "tumbonas" : "no disponible",
        "elementos básicos para la playatoallas" : "no disponible"
    },

    "vistas panorámicas" : {
       "vistas a la playa" : "vistas al mar",
        "vistas al puerto" : "vistas al mar"
    }

}

def clean_service_by_category(value, category):
    if isinstance(value,str): # Aplicamos solo si es str

        if category in cleaning_rules: # Miramos si la categoria esta en el diccionario 
            for key, replacement in cleaning_rules[category].items(): # Terminos dentro de la categoria
                if key in value:
                    return replacement
        return value # Si no se encuentra nada, retorna el valor
    return np.nan # Si el value no es str, devolverá nan
           


In [70]:
# Miramos todas las categorias
category_service_counts.keys()

dict_keys(['baño', 'dormitorio y lavandería', 'entretenimiento', 'calefacción y refrigeración', 'seguridad en el hogar', 'internet y oficina', 'cocina y comedor', 'características de la ubicación', 'exterior', 'aparcamiento e instalaciones', 'servicios', 'no incluidos', 'privacidad y seguridad', 'para familias', 'vistas panorámicas'])

In [71]:
# Usar apply para aplicar funcion al df
# lambda toma como argumento una fila(row) donde pasara los valores de 'Service' y 'Category' y axis = 1 hace referencia a las filas.
df_servicios_cleaned['services'] = df_servicios_cleaned.apply(lambda row: clean_service_by_category(row['service'], row['category']), axis = 1) 

C:\Users\jesus\AppData\Local\Temp\ipykernel_20116\2267139165.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_servicios_cleaned['services'] = df_servicios_cleaned.apply(lambda row: clean_service_by_category(row['service'], row['category']), axis = 1)


In [72]:
clean_category_service_counts = {}

for category in df_servicios_cleaned['category'].unique():
    
    # Filtrar el DataFrame para la categoría actual y contar los servicios
    clean_service_count = df_servicios_cleaned[df_servicios_cleaned['category'] == category]['services'].value_counts()

    # Agregar al diccionario el count
    clean_category_service_counts[category] = clean_service_count


In [73]:
clean_category_service_counts.keys()

dict_keys(['baño', 'dormitorio y lavandería', 'entretenimiento', 'calefacción y refrigeración', 'seguridad en el hogar', 'internet y oficina', 'cocina y comedor', 'características de la ubicación', 'exterior', 'aparcamiento e instalaciones', 'servicios', 'no incluidos', 'privacidad y seguridad', 'para familias', 'vistas panorámicas'])

### Eliminar filas que contengan no disponible

In [75]:
# Eliminar filas donde 'Clean Service' contiene 'no disponible'
df_servicios_cleaned = df_servicios_cleaned[~df_servicios_cleaned['services'].str.contains("no disponible", case=False, na=False)]

df_servicios_cleaned

,category,service,urls,services
0,baño,secador de pelo,https://www.airbnb.es/rooms/126311759129279497...,secador de pelo
1,baño,productos de limpieza,https://www.airbnb.es/rooms/126311759129279497...,productos de limpieza
2,baño,champú,https://www.airbnb.es/rooms/126311759129279497...,champú
3,baño,gel de ducha,https://www.airbnb.es/rooms/126311759129279497...,gel de ducha
4,baño,agua caliente,https://www.airbnb.es/rooms/126311759129279497...,agua caliente
...,...,...,...,...
90741,aparcamiento e instalaciones,ascensorel alojamiento o edificio dispone de u...,https://www.airbnb.es/rooms/14352069?adults=1&...,ascensor
90742,aparcamiento e instalaciones,aparcamiento de pago fuera de las instalaciones,https://www.airbnb.es/rooms/14352069?adults=1&...,aparcamiento
90743,servicios,disponible para estancias largaspermite estanc...,https://www.airbnb.es/rooms/14352069?adults=1&...,disponible para estancias largaspermite estanc...
90744,servicios,llegada autónoma,https://www.airbnb.es/rooms/14352069?adults=1&...,llegada autónoma


In [76]:
df_servicios_cleaned = df_servicios_cleaned[df_servicios_cleaned['category'] != 'vistas panorámicas']

In [77]:
df_servicios_cleaned = df_servicios_cleaned.reset_index(drop = True)
df_servicios_cleaned

,category,service,urls,services
0,baño,secador de pelo,https://www.airbnb.es/rooms/126311759129279497...,secador de pelo
1,baño,productos de limpieza,https://www.airbnb.es/rooms/126311759129279497...,productos de limpieza
2,baño,champú,https://www.airbnb.es/rooms/126311759129279497...,champú
3,baño,gel de ducha,https://www.airbnb.es/rooms/126311759129279497...,gel de ducha
4,baño,agua caliente,https://www.airbnb.es/rooms/126311759129279497...,agua caliente
...,...,...,...,...
75051,aparcamiento e instalaciones,ascensorel alojamiento o edificio dispone de u...,https://www.airbnb.es/rooms/14352069?adults=1&...,ascensor
75052,aparcamiento e instalaciones,aparcamiento de pago fuera de las instalaciones,https://www.airbnb.es/rooms/14352069?adults=1&...,aparcamiento
75053,servicios,disponible para estancias largaspermite estanc...,https://www.airbnb.es/rooms/14352069?adults=1&...,disponible para estancias largaspermite estanc...
75054,servicios,llegada autónoma,https://www.airbnb.es/rooms/14352069?adults=1&...,llegada autónoma


In [78]:
df_servicios_cleaned = df_servicios_cleaned.drop(['service'], axis = 1)
df_servicios_cleaned

,category,urls,services
0,baño,https://www.airbnb.es/rooms/126311759129279497...,secador de pelo
1,baño,https://www.airbnb.es/rooms/126311759129279497...,productos de limpieza
2,baño,https://www.airbnb.es/rooms/126311759129279497...,champú
3,baño,https://www.airbnb.es/rooms/126311759129279497...,gel de ducha
4,baño,https://www.airbnb.es/rooms/126311759129279497...,agua caliente
...,...,...,...
75051,aparcamiento e instalaciones,https://www.airbnb.es/rooms/14352069?adults=1&...,ascensor
75052,aparcamiento e instalaciones,https://www.airbnb.es/rooms/14352069?adults=1&...,aparcamiento
75053,servicios,https://www.airbnb.es/rooms/14352069?adults=1&...,disponible para estancias largaspermite estanc...
75054,servicios,https://www.airbnb.es/rooms/14352069?adults=1&...,llegada autónoma


In [79]:
df_servicios_cleaned.to_csv('data/df_servicios_final_cleaned.csv', index=False)

## Creacion dataframe para hacer encoders

In [81]:
df1 = pd.read_csv('data/df_final_cleaned.csv')
df2 = pd.read_csv('data/df_servicios_final_cleaned.csv')

In [82]:
# Hacemos merge del df de la habitacion con el de servicios
df3 = pd.merge(left = df1, right = df2,how = 'inner',on = "urls")

In [83]:
# Preparamos el df con las variables a analizar
df_services_merged = df3[['urls','prices_per_night','category','services']]

In [84]:
df_table = df_services_merged.groupby(by = ["urls", "category"], as_index = False).agg({"services" : "count"})
df_table

,urls,category,services
0,https://www.airbnb.es/rooms/100044781045691589...,aparcamiento e instalaciones,1
1,https://www.airbnb.es/rooms/100044781045691589...,baño,5
2,https://www.airbnb.es/rooms/100044781045691589...,calefacción y refrigeración,2
3,https://www.airbnb.es/rooms/100044781045691589...,cocina y comedor,13
4,https://www.airbnb.es/rooms/100044781045691589...,dormitorio y lavandería,8
...,...,...,...
21871,https://www.airbnb.es/rooms/999334336680170645...,dormitorio y lavandería,8
21872,https://www.airbnb.es/rooms/999334336680170645...,entretenimiento,2
21873,https://www.airbnb.es/rooms/999334336680170645...,internet y oficina,2
21874,https://www.airbnb.es/rooms/999334336680170645...,seguridad en el hogar,2


In [85]:
# Crear una tabla pivote con categorías como columnas y el número de servicios en cada categoría
df_servicios_pivot = df_table.pivot_table(
    index='urls',         
    columns='category', 
    values='services',   
    aggfunc='sum',
    fill_value=0 
).reset_index()

In [86]:
df_servicios_pivot

category,urls,aparcamiento e instalaciones,baño,calefacción y refrigeración,características de la ubicación,cocina y comedor,dormitorio y lavandería,entretenimiento,exterior,internet y oficina,para familias,privacidad y seguridad,seguridad en el hogar,servicios
0,https://www.airbnb.es/rooms/100044781045691589...,1,5,2,0,13,8,1,0,2,1,0,0,0
1,https://www.airbnb.es/rooms/100051458827470719...,0,2,0,0,2,2,0,0,2,0,1,0,2
2,https://www.airbnb.es/rooms/100114773709033483...,1,5,4,0,13,6,2,1,2,2,0,3,2
3,https://www.airbnb.es/rooms/10038461?adults=1&...,1,2,2,3,9,6,1,2,2,0,0,0,2
4,https://www.airbnb.es/rooms/100569042845880747...,0,0,0,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,https://www.airbnb.es/rooms/998945864657673961...,2,3,3,1,7,5,1,1,1,0,0,2,1
2493,https://www.airbnb.es/rooms/999180200821552889...,1,6,2,0,14,8,1,1,2,0,2,0,4
2494,https://www.airbnb.es/rooms/999188656384515552...,0,5,0,0,10,10,0,0,2,0,4,0,3
2495,https://www.airbnb.es/rooms/999287710180353527...,1,7,1,0,10,8,0,1,2,0,1,0,2


In [87]:
# Hacemos merge del df de la habitacion con el de servicios
df_to_encode = pd.merge(left = df, right = df_servicios_pivot,how = 'inner',on = "urls")
df_to_encode

,urls,timestamp,record_id,titles,host_name,property_types,prices_per_night,location,ratings,num_reviews,...,características de la ubicación,cocina y comedor,dormitorio y lavandería,entretenimiento,exterior,internet y oficina,para familias,privacidad y seguridad,seguridad en el hogar,servicios
0,https://www.airbnb.es/rooms/126311759129279497...,2024-11-07 23:54:16,1263117591292794971,"La ubicación muy buena, cerca de todo si estás...",Carmen,Alojamiento entero,115.0,"Barcelona, Catalunya, España",0.00,0.0,...,2,6,7,1,2,1,0,0,1,4
1,https://www.airbnb.es/rooms/127904085557632410...,2024-11-07 23:54:16,1279040855576324107,Este anfitrión tiene 979 evaluaciones de otros...,David,Habitación,46.0,"Barcelona, Catalunya, España",0.00,0.0,...,0,10,6,0,0,2,0,2,0,1
2,https://www.airbnb.es/rooms/913187918206344111...,2024-11-07 23:54:16,913187918206344111,Esta acogedora habitación cuenta con una cómod...,Manuel,Habitación,47.0,NaN,4.66,65.0,...,2,11,8,1,1,1,0,1,0,5
3,https://www.airbnb.es/rooms/126566083301808951...,2024-11-07 23:54:16,1265660833018089515,Instalaciones y servicios nuevos a estrenar.es...,BLAU Student Housing,Alojamiento entero,100.0,"Barcelona, Catalunya, España",4.89,0.0,...,1,10,10,10,3,2,2,0,5,4
4,https://www.airbnb.es/rooms/31977850?adults=1&...,2024-11-07 23:54:16,31977850,hay una razón por la que hemos sido votados co...,St Christophers Barcelona,Habitación,33.0,"Barcelona, España",4.40,990.0,...,0,3,4,0,1,1,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,https://www.airbnb.es/rooms/13151223?adults=1&...,2024-11-07 23:54:16,13151223,Una habitación muy tranquila para 2-3 personas...,Mike,Habitación,55.0,NaN,4.74,197.0,...,0,8,7,1,0,2,0,3,0,1
2493,https://www.airbnb.es/rooms/109718701780399279...,2024-11-07 23:54:16,1097187017803992791,Olvídate de las preocupaciones en este alojami...,Beverly,Habitación,60.0,"Barcelona, Catalunya, España",4.78,59.0,...,0,8,4,1,0,1,0,1,0,1
2494,https://www.airbnb.es/rooms/20975351?adults=1&...,2024-11-07 23:54:16,20975351,"Piso tipo dúplex de obra nueva, muy luminoso, ...",Aylén,Alojamiento entero,104.0,NaN,4.96,0.0,...,3,16,9,1,0,2,0,0,2,3
2495,https://www.airbnb.es/rooms/52528580?adults=1&...,2024-11-07 23:54:16,52528580,Precioso estudio muy cerca de la playa muy tra...,Gustavo,Alojamiento entero,120.0,NaN,4.83,136.0,...,4,13,8,3,3,2,0,0,2,1


## Convertir horas en minutos

In [89]:
# Convertir la columna a formato tiempo
df_to_encode['check_out_hour'] = pd.to_datetime(df_to_encode['check_out_hour'], format='%H:%M')

# Calcular los minutos desde la medianoche
df_to_encode['check_out_hour'] = df_to_encode['check_out_hour'].dt.hour * 60 + df_to_encode['check_out_hour'].dt.minute


In [90]:
# Convertir la columna a formato tiempo
df_to_encode['check_in_hour'] = pd.to_datetime(df_to_encode['check_in_hour'], format='%H:%M')

# Calcular los minutos desde la medianoche
df_to_encode['check_in_hour'] = df_to_encode['check_in_hour'].dt.hour * 60 + df_to_encode['check_in_hour'].dt.minute

In [91]:
#for column in df_to_encode.columns:
    #if column in ['check_in_hour', 'check_out_hour']:
        
        # Convertir la columna a formato tiempo
        #df[column] = pd.to_datetime(df[column], format='%H:%M', errors = 'coerce')

        # Calcular los minutos desde la medianoche
        #df_to_encode[column] = df_to_encode[column].dt.hour * 60 + df_to_encode[column].dt.minute

    #else:
        #continue

In [92]:
df_to_encode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2497 entries, 0 to 2496
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   urls                             2497 non-null   object 
 1   timestamp                        2497 non-null   object 
 2   record_id                        2497 non-null   int64  
 3   titles                           2496 non-null   object 
 4   host_name                        2496 non-null   object 
 5   property_types                   2497 non-null   object 
 6   prices_per_night                 2497 non-null   float64
 7   location                         1261 non-null   object 
 8   ratings                          2497 non-null   float64
 9   num_reviews                      2497 non-null   float64
 10  cleaning_fee                     2426 non-null   float64
 11  dormitorios                      1415 non-null   float64
 12  camas               

In [93]:
df_to_encode.isna().sum()

urls                                  0
timestamp                             0
record_id                             0
titles                                1
host_name                             1
property_types                        0
prices_per_night                      0
location                           1236
ratings                               0
num_reviews                           0
cleaning_fee                         71
dormitorios                        1082
camas                               103
baños                                65
maximum_guests                        2
check_in_hour                       192
check_out_hour                      184
total_hours_checkin                 192
aparcamiento e instalaciones          0
baño                                  0
calefacción y refrigeración           0
características de la ubicación       0
cocina y comedor                      0
dormitorio y lavandería               0
entretenimiento                       0


In [94]:
df_to_encode = df_to_encode.drop(['urls', 'timestamp', 'record_id','titles','host_name','location',], axis=1)


In [95]:
df_to_encode.columns

Index(['property_types', 'prices_per_night', 'ratings', 'num_reviews',
       'cleaning_fee', 'dormitorios', 'camas', 'baños', 'maximum_guests',
       'check_in_hour', 'check_out_hour', 'total_hours_checkin',
       'aparcamiento e instalaciones', 'baño', 'calefacción y refrigeración',
       'características de la ubicación', 'cocina y comedor',
       'dormitorio y lavandería', 'entretenimiento', 'exterior',
       'internet y oficina', 'para familias', 'privacidad y seguridad',
       'seguridad en el hogar', 'servicios'],
      dtype='object')

In [96]:
df_to_encode

,property_types,prices_per_night,ratings,num_reviews,cleaning_fee,dormitorios,camas,baños,maximum_guests,check_in_hour,...,características de la ubicación,cocina y comedor,dormitorio y lavandería,entretenimiento,exterior,internet y oficina,para familias,privacidad y seguridad,seguridad en el hogar,servicios
0,Alojamiento entero,115.0,0.00,0.0,0.0,1.0,1.0,1.0,NaN,900.0,...,2,6,7,1,2,1,0,0,1,4
1,Habitación,46.0,0.00,0.0,15.0,NaN,1.0,0.5,1.0,1020.0,...,0,10,6,0,0,2,0,2,0,1
2,Habitación,47.0,4.66,65.0,0.0,NaN,1.0,0.5,1.0,900.0,...,2,11,8,1,1,1,0,1,0,5
3,Alojamiento entero,100.0,4.89,0.0,35.0,1.0,1.0,1.0,1.0,960.0,...,1,10,10,10,3,2,2,0,5,4
4,Habitación,33.0,4.40,990.0,0.0,NaN,1.0,0.5,1.0,900.0,...,0,3,4,0,1,1,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,Habitación,55.0,4.74,197.0,10.0,NaN,1.0,0.5,3.0,900.0,...,0,8,7,1,0,2,0,3,0,1
2493,Habitación,60.0,4.78,59.0,0.0,NaN,1.0,0.5,2.0,900.0,...,0,8,4,1,0,1,0,1,0,1
2494,Alojamiento entero,104.0,4.96,0.0,0.0,2.0,3.0,2.0,4.0,900.0,...,3,16,9,1,0,2,0,0,2,3
2495,Alojamiento entero,120.0,4.83,136.0,50.0,1.0,1.0,1.0,2.0,900.0,...,4,13,8,3,3,2,0,0,2,1


In [97]:
df_to_encode.to_csv('data/df_to_encode.csv', index = False )